In [17]:
pip install firecrawl-py pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.9 MB/s eta 0:00:00


In [18]:
from firecrawl import FirecrawlApp
import pandas as pd
from sqlalchemy import create_engine

In [19]:
url = 'https://www.nielsen.com/data-center/top-ten/#streaming'

In [21]:
api_key = 'fc-1b6072619f9941ae9732a3b1a4d64551'  # <-- Paste your API key here
app = FirecrawlApp(api_key)

In [24]:
scrape_result = app.scrape_url(
    url,
    formats=['json'],
    jsonOptions={
        'prompt': 'Extract a structured JSON table of the top streaming shows, including rank, program name, streaming platform, and total minutes viewed.'
    }
)

In [27]:
streaming_data = scrape_result.json

print(streaming_data)

{'topStreamingShows': [{'rank': 1, 'programName': 'Adolescence', 'streamingPlatform': 'Netflix', 'totalMinutesViewed': 1442000}, {'rank': 2, 'programName': 'The Residence', 'streamingPlatform': 'Netflix', 'totalMinutesViewed': 1355000}, {'rank': 3, 'programName': 'Moana 2', 'streamingPlatform': 'Disney+', 'totalMinutesViewed': 1129000}, {'rank': 4, 'programName': 'Reacher', 'streamingPlatform': 'Prime Video', 'totalMinutesViewed': 1097000}, {'rank': 5, 'programName': "Grey's Anatomy", 'streamingPlatform': 'Hulu / Netflix', 'totalMinutesViewed': 977000}, {'rank': 6, 'programName': 'Bluey', 'streamingPlatform': 'Disney+', 'totalMinutesViewed': 975000}, {'rank': 7, 'programName': 'The White Lotus', 'streamingPlatform': 'Max', 'totalMinutesViewed': 973000}, {'rank': 8, 'programName': 'Family Guy', 'streamingPlatform': 'Hulu', 'totalMinutesViewed': 919000}, {'rank': 9, 'programName': 'Wicked', 'streamingPlatform': 'Peacock', 'totalMinutesViewed': 905000}, {'rank': 10, 'programName': 'Severa

In [28]:
streaming_data

{'topStreamingShows': [{'rank': 1,
   'programName': 'Adolescence',
   'streamingPlatform': 'Netflix',
   'totalMinutesViewed': 1442000},
  {'rank': 2,
   'programName': 'The Residence',
   'streamingPlatform': 'Netflix',
   'totalMinutesViewed': 1355000},
  {'rank': 3,
   'programName': 'Moana 2',
   'streamingPlatform': 'Disney+',
   'totalMinutesViewed': 1129000},
  {'rank': 4,
   'programName': 'Reacher',
   'streamingPlatform': 'Prime Video',
   'totalMinutesViewed': 1097000},
  {'rank': 5,
   'programName': "Grey's Anatomy",
   'streamingPlatform': 'Hulu / Netflix',
   'totalMinutesViewed': 977000},
  {'rank': 6,
   'programName': 'Bluey',
   'streamingPlatform': 'Disney+',
   'totalMinutesViewed': 975000},
  {'rank': 7,
   'programName': 'The White Lotus',
   'streamingPlatform': 'Max',
   'totalMinutesViewed': 973000},
  {'rank': 8,
   'programName': 'Family Guy',
   'streamingPlatform': 'Hulu',
   'totalMinutesViewed': 919000},
  {'rank': 9,
   'programName': 'Wicked',
   'str

In [29]:
type(streaming_data)

dict

In [31]:
df_streaming = pd.DataFrame(streaming_data)
df_streaming.head()

,topStreamingShows
0,"{'rank': 1, 'programName': 'Adolescence', 'str..."
1,"{'rank': 2, 'programName': 'The Residence', 's..."
2,"{'rank': 3, 'programName': 'Moana 2', 'streami..."
3,"{'rank': 4, 'programName': 'Reacher', 'streami..."
4,"{'rank': 5, 'programName': 'Grey's Anatomy', '..."


In [32]:
df = pd.DataFrame(streaming_data)

In [33]:
df

,topStreamingShows
0,"{'rank': 1, 'programName': 'Adolescence', 'str..."
1,"{'rank': 2, 'programName': 'The Residence', 's..."
2,"{'rank': 3, 'programName': 'Moana 2', 'streami..."
3,"{'rank': 4, 'programName': 'Reacher', 'streami..."
4,"{'rank': 5, 'programName': 'Grey's Anatomy', '..."
5,"{'rank': 6, 'programName': 'Bluey', 'streaming..."
6,"{'rank': 7, 'programName': 'The White Lotus', ..."
7,"{'rank': 8, 'programName': 'Family Guy', 'stre..."
8,"{'rank': 9, 'programName': 'Wicked', 'streamin..."
9,"{'rank': 10, 'programName': 'Severance', 'stre..."


In [40]:
df_clean = pd.json_normalize(df_streaming['topStreamingShows'])

df_clean.head()

df_clean.columns = ['Rank', 'Program', 'Platform', 'Minutes Viewed']
df_clean.head()

pd.set_option('display.max_rows', 20)

df_clean

,Rank,Program,Platform,Minutes Viewed
0,1,Adolescence,Netflix,1442000
1,2,The Residence,Netflix,1355000
2,3,Moana 2,Disney+,1129000
3,4,Reacher,Prime Video,1097000
4,5,Grey's Anatomy,Hulu / Netflix,977000
5,6,Bluey,Disney+,975000
6,7,The White Lotus,Max,973000
7,8,Family Guy,Hulu,919000
8,9,Wicked,Peacock,905000
9,10,Severance,Apple TV+,876000


In [41]:
user = 'admin'
password = 'isba_4715'
host = 'isba-dev-01.ctwgc6gcobls.us-east-2.rds.amazonaws.com'
port = '3306'
database = 'streaming_data'

In [42]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

In [43]:
df_clean.to_sql(
    name='nielsen_streaming_rankings',
    con=engine,
    if_exists='replace',
    index=False
)

10